# 00 - Налаштування середовища

---
## Налаштування

вхідні дані: 

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
REGION = 'us-central1'

пакети:

In [ ]:
from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

клієнти:

In [ ]:
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

параметри:

In [ ]:
BUCKET = PROJECT_ID

---
## Створити кошик для зберігання
Перевіряє, чи вже існує кошик, і створює його, якщо він відсутній:
- [GCS Python Client](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client)

In [ ]:
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Створення кошика: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Кошик вже створений: {bucketDef.name}')

In [ ]:
print(f'Ознайомитися зі сховищем у консолі можна тут:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID};tab=objects&project={PROJECT_ID}')

---
<a id = 'permissions'></a>
## Обліковий запис і дозволи

Цей екземпляр ноутбука запущено як службовий обліковий запис у GCP.  Цей службовий обліковий запис також використовуватиметься для запуску інших служб у Vertex AI, таких як навчальні завдання та конвеєри.  Обліковий запис служби потребує дозволу на взаємодію з об'єктом у хмарному сховищі, для чого потрібна роль ([roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles)).

Отримайте поточний сервісний обліковий запис:

In [ ]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

Увімкніть API Cloud Resource Manager:

In [ ]:
!gcloud services enable cloudresourcemanager.googleapis.com

Перерахуйте поточні ролі службових акаунтів:

In [ ]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

Якщо в отриманому списку відсутня роль `roles/storage.objectAdmin` або інша роль, яка містить цей дозвіл, наприклад, базова роль `roles/owner`, то її потрібно буде додати для облікового запису служби. Скористайтеся цими інструкціями, щоб зробити це:

In [ ]:
print(f'Перейдіть до IAM в Google Cloud Console:\nhttps://console.cloud.google.com/iam-admin/iam?orgonly=true&project={PROJECT_ID}&supportedpurview=organizationId')

За посиланням на консоль вище або перейшовши за адресою https:/console.cloud.google.com і перейшовши до "IAM & Admin > IAM":
- Знайдіть рядок для облікового запису служби, перерахованого вище: `<номер проекту>-compute@developer.gserviceaccount.com`.
- У колонці `inheritance` натисніть на іконку олівця для редагування ролей
- У спливаючому меню в розділі `Assign roles` виберіть `Add Another Role`.
- Натисніть на поле `Select a role` і введіть `Storage Object Admin`, потім виберіть `Storage Object Admin`.
- Натисніть кнопку Зберегти
- Повторно запустіть список служб нижче і переконайтеся, що роль було додано:

In [ ]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

---
## Встановіть KFP
Якщо ви отримуєте помилку після якогось кроку, повторіть його.  Іноді залежні види вирішуються.
- [Встановлення Kubeflow Pipelines SDK](https://www.kubeflow.org/docs/components/pipelines/v1/sdk/install-sdk/)

In [ ]:
!pip install kfp -U -q

In [ ]:
!pip install google-cloud-pipeline-components -U -q

---
## Оновити пакет AIPlatform:

Пакет `google-cloud-aiplatform` часто оновлюється.  Оновіть його, щоб отримати найновішу функціональність.

- [aiplatform Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
- [GitHub Repo для api-common-protos](https://github.com/googleapis/api-common-protos)

In [ ]:
!pip install googleapis-common-protos -U -q

In [ ]:
!pip install google-cloud-aiplatform -U -q

In [ ]:
from google.cloud import aiplatform
aiplatform.__version__